
---

# MODO DE USO (local o Colab)

## Nuevo modelo desde cero

1. **Añade el dataset**  
    Coloca en la misma carpeta que este `.ipynb` el archivo `entrenamiento_modelo.zip` con la estructura:
    ```
    entrenamiento_modelo/
    ├── train/
    │   ├── images/
    │   └── labels/
    ├── valid/
    │   ├── images/
    │   └── labels/
    ├── test/
    │   ├── images/
    │   └── labels/
    └── data.yaml
    ```

2. **Ejecuta las celdas en orden:**
    - Celda 0: Imports
    - Celda 1: Configuración Entorno (Muestra GPU local)
    - Celda 2: Instala dependencias
    - Celda 3: Prepara el dataset (desde un .zip)
    - Celda 4: Configura el `data.yaml` (si no lo encuentra lo crea, modifícalo a mano para tu dataset)
    - Celda 4.5: Diagnóstico de posibles problemas
    - Celda 5: Visualizar algunas muestras (opcional)
    - Celda 6: Sugerencias de configuración del entrenamiento
    - Celda 7: Entrenamiento con Transfer Learning  
      **Ejemplo de ejecución:**
      ```python
      modelo_entrenado, resultados = entrenar_detector_huevos(
            model_size='n',  # Cambia según tu preferencia: 'n', 's', 'm', 'l', 'x'
            epochs=30,       # Ajusta según tiempo disponible
            batch_size=32,   # Ajusta según GPU disponible
            img_size=640,
            patience=20
      )
      ```
      > Modifica los parámetros de "# Data Augmentation específica para huevos" según tu dataset.
    - Celda 8: Evaluación y métricas del modelo recién creado (Lo normal es que busque y coja automáticamente el último modelo creado/mejorado)
    - Celda 9: Prueba modelo creado con imágenes nuevas (Lo normal es que busque y coja automáticamente el último modelo creado/mejorado)
    - Celda 10: Continuación del entrenamiento (mejora y compara con el anterior, la comparación a veces no funciona al mostrar la tabla)
    - Celda 11: Consejos

---

## Mejorar modelo ya creado aquí

1. **Ejecuta las celdas en orden:**
    - Celda Inicial: Imports
    - Celda 1: Configuración Entorno (Muestra GPU local)
    - Celda 2: Instala dependencias
    - Celda 4.5: Diagnóstico de posibles problemas
    - Celda 6: Sugerencias de configuración del entrenamiento (opcional)
    - Celda 10: Continuación del entrenamiento (mejora y compara con el anterior, la comparación a veces no funciona al mostrar la tabla)

2. **Extra:**  
    - Celda 9: Prueba modelo mejorado con imágenes nuevas (Lo normal es que busque y coja automáticamente el último modelo creado/mejorado)

---

## SOLUCIÓN DE ERRORES

- Si al crear o continuar entrenando un modelo la GPU se queda sin memoria:
  1. Borra la carpeta nueva de `trainX` en `runs/detect`.
  2. En una terminal, ejecuta `nvidia-smi` para ver los procesos que usan GPU.
  3. Mata el proceso con `kill NUMERO_PROCESO` (por ejemplo, `kill 11939`).
  4. Reinicia el kernel si Visual Studio lo solicita.
  5. Vuelve a ejecutar los pasos desde cero, **cambiando** el `batch_size` o `img_size` a valores menores.
  6. Deja al menos 1GB de VRAM libre (por ejemplo, en una RTX 3060 de 6GB, no uses más de 5GB por epoch).
- Si al intentar evaluar un modelo te dice que no lo encuentra:
  1. Vete a `runs/` y busca la última `train/weights` y coge el `best.pt`
  2. Pega el modelo en la carpeta que te pida
  3. Lo normal es que coja automáticamente el último modelo creado/mejorado

---

**Consejo:**  
Evita usar bloques de código para instrucciones largas, usa listas y encabezados para mayor claridad y mejor visualización en Jupyter/Colab.

In [ ]:
# CELDA 0: IMPORTS NECESARIOS AL INICIO

# YOLOv8 Transfer Learning - Detector de Huevos Brown/White
# Optimizado para entrenamiento local con RTX 3060 Laptop 6GB

import os
import shutil
import torch
from pathlib import Path
import zipfile
import cv2
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image, display
import random


In [ ]:
# CELDA 1: Configuración inicial y verificación de GPU

# Verificar GPU disponible
import torch
import os
from pathlib import Path

print("🔧 CONFIGURACIÓN DEL ENTORNO")
print("=" * 50)

# Verificar GPU
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU disponible: {gpu_name}")
    print(f"✅ CUDA Version: {torch.version.cuda}")
    print(f"✅ Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No hay GPU disponible - El entrenamiento será muy lento")

print(f"✅ PyTorch version: {torch.__version__}")

In [ ]:
# CELDA 2: Instalación de dependencias

# Instalar Ultralytics (YOLOv8)
!pip install ultralytics roboflow

# Imports necesarios
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image, display
import shutil
import zipfile

print("📦 Dependencias instaladas correctamente")

In [ ]:
# CELDA 3: Preparar y descomprimir dataset

print("📁 PREPARACIÓN DEL DATASET")
print("=" * 50)

# Función mejorada para subir y procesar dataset
def subir_y_procesar_dataset():
    """Procesar dataset desde archivos subidos en Colab"""
    print("📁 Buscando archivos ZIP subidos en Colab...")

    # Buscar archivos ZIP en el directorio actual
    zip_files = [f for f in os.listdir('.') if f.endswith('.zip')]

    if not zip_files:
        print("❌ No se encontraron archivos ZIP")
        print("💡 Asegúrate de subir tu dataset ZIP usando el panel de archivos de Colab")
        return False

    # Usar el primer ZIP encontrado
    zip_name = zip_files[0]
    print(f"✅ Archivo ZIP encontrado: {zip_name}")

    # Limpiar directorio anterior si existe
    if os.path.exists('dataset'):
        shutil.rmtree('dataset')

    # Descomprimir
    try:
        with zipfile.ZipFile(zip_name, 'r') as zip_ref:
            zip_ref.extractall('.')
        print("✅ Dataset descomprimido")
    except Exception as e:
        print(f"❌ Error al descomprimir: {e}")
        return False

    # Detectar estructura y reorganizar si es necesario
    return detectar_y_reorganizar_estructura()


def detectar_y_reorganizar_estructura():
    """Detecta la estructura del dataset y la reorganiza si es necesario"""
    print("\n🔍 DETECTANDO ESTRUCTURA DEL DATASET...")

    # Buscar posibles ubicaciones del dataset
    posibles_rutas = [
        'entrenamiento_modelo',  # Si se subió la carpeta completa
        '.',  # Si se descomprimió en la raíz
        'dataset'  # Si ya estaba en dataset
    ]

    dataset_path = None
    for ruta in posibles_rutas:
        if os.path.exists(ruta):
            # Verificar si contiene train/valid
            if (os.path.exists(os.path.join(ruta, 'train', 'images')) and
                os.path.exists(os.path.join(ruta, 'valid', 'images'))):
                dataset_path = ruta
                print(f"✅ Dataset encontrado en: {dataset_path}")
                break

    if not dataset_path:
        print("❌ No se encontró estructura válida del dataset")
        print("Explorando directorios disponibles...")
        for root, dirs, files in os.walk('.'):
            if 'images' in dirs and 'labels' in dirs:
                print(f"Posible dataset en: {root}")
        return False

    # Si el dataset no está en 'dataset/', moverlo ahí
    if dataset_path != 'dataset':
        if os.path.exists('dataset'):
            shutil.rmtree('dataset')
        shutil.move(dataset_path, 'dataset')
        print(f"📁 Dataset movido a 'dataset/'")

    return verificar_estructura_dataset()

# Función para verificar estructura del dataset
def verificar_estructura_dataset():
    """Verificar y mostrar la estructura del dataset"""
    base_path = Path('dataset')
    required_structure = [
        'train/images',
        'train/labels',
        'valid/images',
        'valid/labels'
    ]

    print("\n🔍 VERIFICANDO ESTRUCTURA DEL DATASET:")
    print("-" * 40)

    all_good = True
    for item in required_structure:
        full_path = base_path / item
        if full_path.exists():
            if full_path.is_dir():
                # Contar archivos (excluyendo archivos ocultos)
                files_list = [f for f in full_path.iterdir() if f.is_file() and not f.name.startswith('.')]
                count = len(files_list)
                print(f"✅ {item}: {count} archivos")

                # Mostrar algunos ejemplos
                if count > 0:
                    ejemplos = [f.name for f in files_list[:3]]
                    print(f"   Ejemplos: {', '.join(ejemplos)}")
            else:
                print(f"✅ {item}: encontrado (archivo)")
        else:
            print(f"❌ {item}: NO encontrado")
            all_good = False

    # Verificar data.yaml
    yaml_files = list(Path('dataset').glob('*.yaml')) + list(Path('.').glob('data.yaml'))
    if yaml_files:
        print(f"✅ Archivo YAML encontrado: {yaml_files[0]}")
    else:
        print("⚠️ No se encontró data.yaml - se creará automáticamente")

    if all_good:
        print("\n🎉 Estructura del dataset es correcta!")
    else:
        print("\n⚠️ Hay problemas con la estructura del dataset")
        print("Explorando toda la estructura disponible...")
        explorar_estructura_completa()

    return all_good

def explorar_estructura_completa():
    """Explorar toda la estructura de archivos para debug"""
    print("\n🔍 EXPLORANDO ESTRUCTURA COMPLETA:")
    print("-" * 30)

    for root, dirs, files in os.walk('.'):
        # Saltar directorios del sistema
        if any(skip in root for skip in ['.config', '__pycache__', '.git', 'sample_data']):
            continue

        level = root.replace('.', '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")

        # Mostrar solo algunos archivos para no saturar
        subindent = ' ' * 2 * (level + 1)
        for file in files[:5]:  # Solo primeros 5 archivos
            print(f"{subindent}{file}")
        if len(files) > 5:
            print(f"{subindent}... y {len(files) - 5} archivos más")

# Ejecutar y procesar salida

print("🚀 PROCESANDO DATASET DESDE ARCHIVOS DE COLAB:")
print("1. Sube tu archivo ZIP usando el panel de archivos (📁) en la barra lateral")
print("2. El archivo debe contener la estructura:")
print("entrenamiento_modelo/")
print("├── train/")
print("│   ├── images/")
print("│   └── labels/")
print("├── valid/")
print("│   ├── images/")
print("│   └── labels/")
print("└── data.yaml")
print()

# Procesar automáticamente si hay ZIP disponible
if not subir_y_procesar_dataset():
    print("❌ Error al procesar dataset")
    print("💡 Sube tu archivo ZIP y vuelve a ejecutar esta celda")
else:
    print("✅ Dataset procesado correctamente")


In [ ]:
# CELDA 4: Configuración del data.yaml

def crear_data_yaml_dinamico():
    """Crear data.yaml con rutas correctas detectadas automáticamente"""

    # Verificar que existe la estructura del dataset
    if not os.path.exists('dataset/train/images'):
        print("❌ No se encuentra dataset/train/images")
        return False

    # Contar imágenes para verificar
    train_images = len([f for f in Path('dataset/train/images').iterdir() if f.suffix.lower() in ['.jpg', '.jpeg', '.png']])
    val_images = len([f for f in Path('dataset/valid/images').iterdir() if f.suffix.lower() in ['.jpg', '.jpeg', '.png']])

    print(f"📊 Imágenes encontradas:")
    print(f"  Entrenamiento: {train_images}")
    print(f"  Validación: {val_images}")

    # Crear data.yaml con rutas absolutas para evitar problemas
    yaml_content = f"""# Dataset configuration for YOLOv8 Egg Detection
# Paths - usando rutas absolutas para Colab

train: /content/dataset/train/images
val: /content/dataset/valid/images

nc: 2
names: ['Brown Egg', 'White Egg']

# Dataset info
roboflow:
  workspace: data-science-4tmzt
  project: egg-segmentation-in-poultry-industry-based-on-color-pxcll
  version: 2
  license: CC BY 4.0
"""

    with open('data.yaml', 'w') as f:
        f.write(yaml_content)

    print("✅ data.yaml creado con rutas absolutas")

    # Verificar que las rutas existen
    train_path = Path('/content/dataset/train/images')
    val_path = Path('/content/dataset/valid/images')

    if train_path.exists() and val_path.exists():
        print("✅ Rutas verificadas correctamente")
        return True
    else:
        print("❌ Error: Las rutas en data.yaml no existen")
        return False

# Crear data.yaml
if not crear_data_yaml_dinamico():
    print("❌ Error al crear data.yaml")
    print("Creando data.yaml con rutas relativas como respaldo...")

    yaml_content_backup = """# Dataset configuration - rutas relativas
train: dataset/train/images
val: dataset/valid/images

nc: 2
names: ['Brown Egg', 'White Egg']
"""

    with open('data.yaml', 'w') as f:
        f.write(yaml_content_backup)

    print("✅ data.yaml de respaldo creado")



In [ ]:
# CELDA 4.5: Diagnóstico y solución de problemas

def diagnosticar_dataset():
    """Diagnóstico completo del dataset para resolver problemas"""
    print("🔍 DIAGNÓSTICO COMPLETO DEL DATASET")
    print("=" * 50)

    # 1. Verificar estructura actual
    print("1. ESTRUCTURA ACTUAL:")
    current_dirs = []
    for item in os.listdir('.'):
        if os.path.isdir(item) and not item.startswith('.'):
            current_dirs.append(item)
    print(f"Directorios disponibles: {current_dirs}")

    # 2. Buscar directorios con imágenes
    print("\n2. BUSCANDO DIRECTORIOS CON IMÁGENES:")
    image_dirs = []
    for root, dirs, files in os.walk('.'):
        if any(f.lower().endswith(('.jpg', '.jpeg', '.png')) for f in files):
            img_count = len([f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            if img_count > 0:
                image_dirs.append((root, img_count))
                print(f"  {root}: {img_count} imágenes")

    # 3. Buscar directorios con labels
    print("\n3. BUSCANDO DIRECTORIOS CON LABELS:")
    label_dirs = []
    for root, dirs, files in os.walk('.'):
        if any(f.lower().endswith('.txt') for f in files):
            txt_count = len([f for f in files if f.lower().endswith('.txt')])
            if txt_count > 0:
                label_dirs.append((root, txt_count))
                print(f"  {root}: {txt_count} archivos .txt")

    # 4. Intentar reorganizar automáticamente
    print("\n4. REORGANIZACIÓN AUTOMÁTICA:")
    return reorganizar_automaticamente(image_dirs, label_dirs)

def reorganizar_automaticamente(image_dirs, label_dirs):
    """Reorganizar automáticamente la estructura del dataset"""

    # Limpiar dataset anterior
    if os.path.exists('dataset'):
        shutil.rmtree('dataset')
    os.makedirs('dataset', exist_ok=True)

    # Buscar directorios train y valid
    train_img_dir = None
    train_lbl_dir = None
    valid_img_dir = None
    valid_lbl_dir = None

    # Identificar directorios por nombre
    for img_dir, count in image_dirs:
        if 'train' in img_dir.lower() and 'image' in img_dir.lower():
            train_img_dir = img_dir
            print(f"✅ Train images encontrado: {img_dir} ({count} imágenes)")
        elif 'valid' in img_dir.lower() and 'image' in img_dir.lower():
            valid_img_dir = img_dir
            print(f"✅ Valid images encontrado: {img_dir} ({count} imágenes)")

    for lbl_dir, count in label_dirs:
        if 'train' in lbl_dir.lower() and 'label' in lbl_dir.lower():
            train_lbl_dir = lbl_dir
            print(f"✅ Train labels encontrado: {lbl_dir} ({count} labels)")
        elif 'valid' in lbl_dir.lower() and 'label' in lbl_dir.lower():
            valid_lbl_dir = lbl_dir
            print(f"✅ Valid labels encontrado: {lbl_dir} ({count} labels)")

    # Verificar que encontramos todo
    if not all([train_img_dir, train_lbl_dir, valid_img_dir, valid_lbl_dir]):
        print("❌ No se pudieron encontrar todos los directorios necesarios")
        print(f"Train images: {train_img_dir}")
        print(f"Train labels: {train_lbl_dir}")
        print(f"Valid images: {valid_img_dir}")
        print(f"Valid labels: {valid_lbl_dir}")
        return False

    # Crear estructura correcta
    os.makedirs('dataset/train/images', exist_ok=True)
    os.makedirs('dataset/train/labels', exist_ok=True)
    os.makedirs('dataset/valid/images', exist_ok=True)
    os.makedirs('dataset/valid/labels', exist_ok=True)

    # Copiar archivos
    try:
        # Copiar imágenes de entrenamiento
        for file in Path(train_img_dir).iterdir():
            if file.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                shutil.copy2(file, 'dataset/train/images/')

        # Copiar labels de entrenamiento
        for file in Path(train_lbl_dir).iterdir():
            if file.suffix.lower() == '.txt':
                shutil.copy2(file, 'dataset/train/labels/')

        # Copiar imágenes de validación
        for file in Path(valid_img_dir).iterdir():
            if file.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                shutil.copy2(file, 'dataset/valid/images/')

        # Copiar labels de validación
        for file in Path(valid_lbl_dir).iterdir():
            if file.suffix.lower() == '.txt':
                shutil.copy2(file, 'dataset/valid/labels/')

        print("✅ Archivos copiados exitosamente")
        return verificar_estructura_dataset()

    except Exception as e:
        print(f"❌ Error al copiar archivos: {e}")
        return False

# Ejecutar diagnóstico si es necesario
print("🔧 DIAGNÓSTICO AUTOMÁTICO")
print("Ejecutando diagnóstico para resolver problemas de estructura...")
if diagnosticar_dataset():
    print("✅ Dataset reorganizado correctamente")
else:
    print("❌ No se pudo reorganizar automáticamente")
    print("Por favor, verifica manualmente la estructura de tu ZIP")


In [ ]:
# CELDA 5: Visualización de muestras del dataset


def visualizar_muestras():
    """Visualizar algunas imágenes del dataset con sus labels"""
    import random

    train_images = list(Path('dataset/train/images').glob('*.jpg'))
    if not train_images:
        train_images = list(Path('dataset/train/images').glob('*.png'))

    if len(train_images) == 0:
        print("❌ No se encontraron imágenes de entrenamiento")
        return

    # Seleccionar 4 imágenes aleatorias
    sample_images = random.sample(train_images, min(4, len(train_images)))

    plt.figure(figsize=(15, 10))

    for i, img_path in enumerate(sample_images):
        plt.subplot(2, 2, i+1)

        # Cargar imagen
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Cargar labels si existen
        label_path = img_path.parent.parent / 'labels' / f"{img_path.stem}.txt"

        if label_path.exists():
            with open(label_path, 'r') as f:
                lines = f.readlines()

            h, w = img.shape[:2]

            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    x_center = float(parts[1]) * w
                    y_center = float(parts[2]) * h
                    width = float(parts[3]) * w
                    height = float(parts[4]) * h

                    # Dibujar bounding box
                    x1 = int(x_center - width/2)
                    y1 = int(y_center - height/2)
                    x2 = int(x_center + width/2)
                    y2 = int(y_center + height/2)

                    color = (255, 0, 0) if class_id == 0 else (0, 255, 0)  # Rojo para Brown, Verde para White
                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

                    label = "Brown Egg" if class_id == 0 else "White Egg"
                    cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        plt.imshow(img)
        plt.title(f"Muestra {i+1}: {img_path.name}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Ejecutar visualización (comentar si no quieres ejecutar)
    visualizar_muestras()

In [ ]:
# CELDA 6: Configuración de entrenamiento optimizada

import torch

# Configuración de modelos disponibles
MODELOS_YOLO = {
    'n': 'yolov8n.pt',  # Nano - 3.2M params - Muy rápido
    's': 'yolov8s.pt',  # Small - 11.2M params - Balance óptimo
    'm': 'yolov8m.pt',  # Medium - 25.9M params - Más preciso
    'l': 'yolov8l.pt',  # Large - 43.7M params - Muy preciso
    'x': 'yolov8x.pt'   # Extra Large - 68.2M params - Máxima precisión
}

def recomendar_configuracion():
    """Configuración optimizada según memoria GPU detectada"""
    if torch.cuda.is_available():
        props = torch.cuda.get_device_properties(0)
        gpu_memory = props.total_memory / 1e9
        gpu_name = props.name.lower()

        # Ajuste específico para RTX 3060 Laptop 6GB
        if "3060" in gpu_name and "laptop" in gpu_name and gpu_memory <= 6.1:
            return {
                'model': MODELOS_YOLO['n'],
                'epochs': 30,
                'batch': 34,  # Confirmado que usa ~5.2GB/6GB
                'imgsz': 640,
                'patience': 20
            }
        elif gpu_memory >= 15:  # T4 o superior
            return {
                'model': MODELOS_YOLO['n'],
                'epochs': 50,
                'batch': 64,
                'imgsz': 640,
                'patience': 20
            }
        elif gpu_memory >= 8:  # GPU media
            return {
                'model': MODELOS_YOLO['n'],
                'epochs': 20,
                'batch': 32,
                'imgsz': 512,
                'patience': 10
            }
        else:  # GPU básica
            return {
                'model': MODELOS_YOLO['n'],
                'epochs': 20,
                'batch': 16,
                'imgsz': 416,
                'patience': 5
            }
    else:  # CPU
        return {
            'model': MODELOS_YOLO['n'],
            'epochs': 5,
            'batch': 4,
            'imgsz': 320,
            'patience': 2
        }

# Ejecutar configuración recomendada
config_recomendada = recomendar_configuracion()
print("🎯 CONFIGURACIÓN RECOMENDADA:")
print(f"Modelo: {config_recomendada['model']}")
print(f"Épocas: {config_recomendada['epochs']}")
print(f"Batch size: {config_recomendada['batch']}")
print(f"Tamaño imagen: {config_recomendada['imgsz']}")


In [ ]:
# CELDA 7: Entrenamiento con Transfer Learning


def entrenar_detector_huevos(
    model_size='s',
    epochs=20,
    batch_size=32,
    img_size=416,
    patience=5
):
    """
    Entrenamiento optimizado con transfer learning para detección de huevos
    """

    print("🚀 INICIANDO ENTRENAMIENTO CON TRANSFER LEARNING")
    print("=" * 60)

    model_path = MODELOS_YOLO[model_size]
    print(f"📦 Cargando modelo base: {model_path}")

    # Cargar modelo preentrenado
    model = YOLO(model_path)

    # Configuración de entrenamiento optimizada para huevos
    training_args = {
        # Dataset
        'data': 'data.yaml',
        'epochs': epochs,
        'imgsz': img_size,
        'batch': batch_size,

        # Output
        'name': f'detector_huevos_{model_size}',
        'project': 'runs/detect',
        'save_period': 5,  # Guardar cada 5 épocas

        # Transfer Learning - Learning rates conservadores
        'lr0': 0.01,        # LR inicial bajo para transfer learning
        'lrf': 0.01,        # LR final
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 3,

        # Data Augmentation específica para huevos
        'hsv_h': 0.015,     # Variación de matiz muy ligera
        'hsv_s': 0.7,       # Saturación importante (brown vs white)
        'hsv_v': 0.4,       # Variación de brillo
        'degrees': 15.0,    # Rotación ligera
        'translate': 0.1,   # Translación
        'scale': 0.5,       # Variación de escala
        'shear': 0.0,       # Sin sesgo (huevos simétricos)
        'perspective': 0.0, # Sin perspectiva extrema
        'flipud': 0.0,      # Sin volteo vertical
        'fliplr': 0.5,      # Volteo horizontal OK
        'mosaic': 1.0,      # Mosaic augmentation
        'mixup': 0.0,       # Sin mixup (confunde colores)
        'copy_paste': 0.0,  # Sin copy-paste

        # Optimización
        'optimizer': 'SGD',  # SGD funciona bien con transfer learning
        'patience': patience,
        'dropout': 0.0,

        # Hardware
        'device': 0 if torch.cuda.is_available() else 'cpu',
        'workers': 2,  # Reducido para Colab

        # Monitoreo
        'verbose': True,
        'plots': True,
        'save': True,
        'val': True,

        # Configuraciones adicionales para Colab
        'cache': False,  # No usar cache para ahorrar memoria
        'amp': True,     # Automatic Mixed Precision para T4
    }

    print("⚙️ Configuración de entrenamiento:")
    for key, value in training_args.items():
        print(f"  {key}: {value}")

    try:
        print(f"\n🔥 Iniciando entrenamiento...")
        results = model.train(**training_args)

        print("\n" + "="*60)
        print("🎉 ENTRENAMIENTO COMPLETADO EXITOSAMENTE")
        print("="*60)

        return model, results

    except Exception as e:
        print(f"❌ Error durante el entrenamiento: {e}")
        print("\n🔄 Intentando con configuración simplificada...")

        # Configuración simplificada
        simple_args = {
            'data': 'data.yaml',
            'epochs': epochs,
            'imgsz': img_size,
            'batch': max(batch_size//2, 4),  # Reducir batch si hay problemas
            'name': f'detector_huevos_{model_size}_simple',
            'lr0': 0.01,
            'patience': patience,
            'device': 0 if torch.cuda.is_available() else 'cpu',
            'amp': False,  # Desactivar AMP si hay problemas
        }

        results = model.train(**simple_args)
        return model, results

# Ejecutar entrenamiento
modelo_entrenado, resultados = entrenar_detector_huevos(
    model_size='n',  # Cambiar según tu preferencia: 'n', 's', 'm', 'l', 'x'
    epochs=30,      # Ajustar según tiempo disponible
    batch_size=32,   # Ajustar según GPU disponible
    img_size=640,
    patience=20
)


In [ ]:
# CELDA 8: Evaluación y métricas

def encontrar_mejor_modelo():
    """Buscar el modelo best.pt en todas las carpetas posibles"""
    runs_dir = Path('runs/detect')
    posibles_rutas = []
    
    if runs_dir.exists():
        # Buscar en todas las subcarpetas de runs/detect
        for carpeta in runs_dir.iterdir():
            if carpeta.is_dir():
                # Buscar best.pt en weights/
                ruta_weights = carpeta / 'weights' / 'best.pt'
                if ruta_weights.exists():
                    posibles_rutas.append((ruta_weights, carpeta.stat().st_mtime))
                
                # También buscar directamente en la carpeta
                ruta_directa = carpeta / 'best.pt'
                if ruta_directa.exists():
                    posibles_rutas.append((ruta_directa, carpeta.stat().st_mtime))
    
    if posibles_rutas:
        # Ordenar por fecha de modificación (más reciente primero)
        posibles_rutas.sort(key=lambda x: x[1], reverse=True)
        return posibles_rutas[0][0]
    
    return None

def evaluar_modelo():
    """Evaluar el modelo entrenado"""
    print("📊 EVALUACIÓN DEL MODELO")
    print("=" * 40)
    
    # Buscar el modelo
    best_model_path = encontrar_mejor_modelo()
    
    if best_model_path is None:
        print("❌ No se encuentra ningún modelo best.pt")
        print("🔍 Verificando carpetas disponibles...")
        runs_dir = Path('runs/detect')
        if runs_dir.exists():
            for carpeta in runs_dir.iterdir():
                if carpeta.is_dir():
                    print(f"  📁 {carpeta.name}/")
                    weights_dir = carpeta / 'weights'
                    if weights_dir.exists():
                        archivos = list(weights_dir.glob('*.pt'))
                        if archivos:
                            print(f"    └── weights/: {[f.name for f in archivos]}")
                        else:
                            print(f"    └── weights/: (vacía)")
                    else:
                        print(f"    └── No hay carpeta weights/")
        return None, None
    
    print(f"✅ Modelo encontrado: {best_model_path}")
    
    try:
        # Cargar modelo
        model = YOLO(str(best_model_path))
        
        # Validación
        print("🔄 Ejecutando validación...")
        metrics = model.val()
        
        print(f"\n📈 MÉTRICAS FINALES:")
        print(f"mAP50: {metrics.box.map50:.3f}")
        print(f"mAP50-95: {metrics.box.map:.3f}")
        print(f"Precision: {metrics.box.mp:.3f}")
        print(f"Recall: {metrics.box.mr:.3f}")
        
        # Mostrar curvas de entrenamiento
        carpeta_modelo = best_model_path.parent.parent
        results_png = carpeta_modelo / 'results.png'
        if results_png.exists():
            print(f"\n📊 Curvas de entrenamiento:")
            display(Image(str(results_png)))
        else:
            print(f"⚠️  No se encontró results.png en {carpeta_modelo}")
            
        return model, metrics
        
    except Exception as e:
        print(f"❌ Error al cargar el modelo: {e}")
        return None, None

# Evaluar modelo
modelo_final, metricas = evaluar_modelo()

In [ ]:
# CELDA 9: Pruebas con imágenes (CORREGIDA) - CON GUARDADO

def probar_modelo_con_muestras():
    """Probar el modelo con imágenes del dataset y guardar resultados"""
    print("🧪 PROBANDO MODELO CON MUESTRAS")
    print("=" * 40)
    
    # Crear carpeta para guardar resultados
    resultados_dir = Path('resultados_pruebas')
    resultados_dir.mkdir(exist_ok=True)
    
    # Buscar modelo
    best_model_path = encontrar_mejor_modelo()
    
    if best_model_path is None:
        print("❌ No se encuentra el modelo best.pt")
        return
    
    print(f"✅ Usando modelo: {best_model_path}")
    
    try:
        model = YOLO(str(best_model_path))
        
        # Buscar imágenes de validación en diferentes ubicaciones posibles
        posibles_dirs = [
            Path('dataset/valid/images'),
            Path('dataset/val/images'),
            Path('valid/images'),
            Path('val/images')
        ]
        
        val_images = []
        for dir_path in posibles_dirs:
            if dir_path.exists():
                # Buscar jpg, png, jpeg
                for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
                    val_images.extend(list(dir_path.glob(ext)))
                if val_images:
                    print(f"📁 Imágenes encontradas en: {dir_path}")
                    break
        
        if not val_images:
            print("❌ No se encontraron imágenes de validación")
            print("🔍 Verificando directorios disponibles...")
            for dir_path in posibles_dirs:
                if dir_path.exists():
                    archivos = list(dir_path.iterdir())
                    print(f"  📁 {dir_path}: {len(archivos)} archivos")
            return
        
        print(f"📸 Total de imágenes disponibles: {len(val_images)}")
        
        # Probar con 6 imágenes aleatorias
        import random
        from datetime import datetime
        
        num_muestras = min(6, len(val_images))
        sample_images = random.sample(val_images, num_muestras)
        
        # Crear nombre único para esta sesión
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        plt.figure(figsize=(20, 15))
        
        # Lista para guardar información de detecciones
        detecciones_info = []
        
        for i, img_path in enumerate(sample_images):
            plt.subplot(2, 3, i+1)
            
            try:
                # Hacer predicción
                results = model(str(img_path), conf=0.25, iou=0.45)
                
                # Visualizar resultado
                annotated = results[0].plot()
                annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
                plt.imshow(annotated_rgb)
                plt.title(f"Predicción: {img_path.name}")
                plt.axis('off')
                
                # Guardar imagen con detecciones
                imagen_guardada = resultados_dir / f"deteccion_{timestamp}_{i+1}_{img_path.stem}.jpg"
                cv2.imwrite(str(imagen_guardada), annotated)
                
                # Recopilar información de detecciones
                imagen_info = {
                    'imagen_original': img_path.name,
                    'imagen_resultado': imagen_guardada.name,
                    'detecciones': []
                }
                
                # Mostrar detecciones en texto
                if results[0].boxes is not None and len(results[0].boxes) > 0:
                    detections = []
                    for box in results[0].boxes:
                        cls = int(box.cls[0])
                        conf = float(box.conf[0])
                        class_name = model.names[cls]
                        detections.append(f"{class_name}: {conf:.2f}")
                        
                        # Guardar coordenadas de la caja
                        xyxy = box.xyxy[0].tolist()  # [x1, y1, x2, y2]
                        imagen_info['detecciones'].append({
                            'clase': class_name,
                            'confianza': round(conf, 3),
                            'coordenadas': [round(x, 1) for x in xyxy]
                        })
                    
                    det_text = "\n".join(detections)
                    plt.text(0.02, 0.98, det_text, transform=plt.gca().transAxes,
                            verticalalignment='top', 
                            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
                else:
                    plt.text(0.02, 0.98, "Sin detecciones", transform=plt.gca().transAxes,
                            verticalalignment='top', 
                            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))
                    imagen_info['detecciones'] = []
                
                detecciones_info.append(imagen_info)
                            
            except Exception as e:
                plt.text(0.5, 0.5, f"Error: {str(e)}", transform=plt.gca().transAxes,
                        ha='center', va='center',
                        bbox=dict(boxstyle='round', facecolor='red', alpha=0.8))
                plt.title(f"Error: {img_path.name}")
                plt.axis('off')
        
        # Guardar el gráfico completo
        grafico_guardado = resultados_dir / f"resumen_pruebas_{timestamp}.png"
        plt.tight_layout()
        plt.savefig(str(grafico_guardado), dpi=300, bbox_inches='tight')
        plt.show()
        
        # Guardar información detallada en JSON
        import json
        info_guardada = resultados_dir / f"detecciones_{timestamp}.json"
        with open(info_guardada, 'w', encoding='utf-8') as f:
            json.dump({
                'timestamp': timestamp,
                'modelo_usado': str(best_model_path),
                'configuracion': {'conf': 0.25, 'iou': 0.45},
                'resultados': detecciones_info
            }, f, indent=2, ensure_ascii=False)
        
        # Crear reporte en texto
        reporte_guardado = resultados_dir / f"reporte_{timestamp}.txt"
        with open(reporte_guardado, 'w', encoding='utf-8') as f:
            f.write(f"REPORTE DE PRUEBAS - {timestamp}\n")
            f.write("="*50 + "\n\n")
            f.write(f"Modelo usado: {best_model_path}\n")
            f.write(f"Configuración: conf=0.25, iou=0.45\n")
            f.write(f"Imágenes probadas: {len(detecciones_info)}\n\n")
            
            for i, info in enumerate(detecciones_info, 1):
                f.write(f"IMAGEN {i}: {info['imagen_original']}\n")
                f.write(f"Resultado guardado: {info['imagen_resultado']}\n")
                if info['detecciones']:
                    f.write(f"Detecciones encontradas: {len(info['detecciones'])}\n")
                    for det in info['detecciones']:
                        f.write(f"  - {det['clase']}: {det['confianza']:.3f} "
                               f"[{', '.join(map(str, det['coordenadas']))}]\n")
                else:
                    f.write("Sin detecciones\n")
                f.write("\n")
        
        print(f"\n💾 RESULTADOS GUARDADOS:")
        print(f"📁 Carpeta: {resultados_dir}/")
        print(f"📊 Gráfico resumen: {grafico_guardado.name}")
        print(f"📄 Reporte detallado: {reporte_guardado.name}")
        print(f"📋 Datos JSON: {info_guardada.name}")
        print(f"🖼️  Imágenes individuales: {len(detecciones_info)} archivos")
        print(f"✅ Pruebas completadas con {num_muestras} imágenes")
        
        return detecciones_info
        
    except Exception as e:
        print(f"❌ Error al cargar el modelo: {e}")
        return None

# Ejecutar pruebas
resultados_pruebas = probar_modelo_con_muestras()


# FUNCIÓN AUXILIAR: Verificar estructura de carpetas

def verificar_estructura():
    """Verificar la estructura de carpetas y archivos"""
    print("🔍 VERIFICACIÓN DE ESTRUCTURA")
    print("=" * 40)
    
    # Verificar runs/detect
    runs_dir = Path('runs/detect')
    if runs_dir.exists():
        print(f"📁 {runs_dir}/")
        for carpeta in sorted(runs_dir.iterdir()):
            if carpeta.is_dir():
                print(f"  ├── {carpeta.name}/")
                
                # Verificar weights
                weights_dir = carpeta / 'weights'
                if weights_dir.exists():
                    archivos_pt = list(weights_dir.glob('*.pt'))
                    if archivos_pt:
                        print(f"  │   ├── weights/")
                        for archivo in archivos_pt:
                            print(f"  │   │   └── {archivo.name}")
                    else:
                        print(f"  │   ├── weights/ (sin archivos .pt)")
                else:
                    print(f"  │   └── (sin carpeta weights)")
                
                # Verificar otros archivos importantes
                archivos_importantes = ['results.png', 'confusion_matrix.png', 'val_batch0_pred.jpg']
                for archivo in archivos_importantes:
                    if (carpeta / archivo).exists():
                        print(f"  │   ├── {archivo}")
    else:
        print("❌ No existe la carpeta runs/detect")
    
    # Verificar dataset
    print(f"\n📁 dataset/")
    dataset_dir = Path('dataset')
    if dataset_dir.exists():
        for subdir in ['train', 'valid', 'val', 'test']:
            subdir_path = dataset_dir / subdir
            if subdir_path.exists():
                images_path = subdir_path / 'images'
                labels_path = subdir_path / 'labels'
                
                num_images = len(list(images_path.glob('*'))) if images_path.exists() else 0
                num_labels = len(list(labels_path.glob('*'))) if labels_path.exists() else 0
                
                print(f"  ├── {subdir}/ ({num_images} imgs, {num_labels} labels)")

# Ejecutar verificación (comentar si no necesitas)
# verificar_estructura()

In [ ]:
# CELDA 10: Continuar entrenamiento desde last.pt

def encontrar_last_modelo():
    """Buscar el modelo last.pt más reciente"""
    runs_dir = Path('runs/detect')
    posibles_rutas = []
    
    if runs_dir.exists():
        for carpeta in runs_dir.iterdir():
            if carpeta.is_dir():
                # Buscar last.pt en weights/
                ruta_weights = carpeta / 'weights' / 'last.pt'
                if ruta_weights.exists():
                    posibles_rutas.append((ruta_weights, carpeta.stat().st_mtime))
                
                # También buscar directamente en la carpeta
                ruta_directa = carpeta / 'last.pt'
                if ruta_directa.exists():
                    posibles_rutas.append((ruta_directa, carpeta.stat().st_mtime))
    
    if posibles_rutas:
        # Ordenar por fecha de modificación (más reciente primero)
        posibles_rutas.sort(key=lambda x: x[1], reverse=True)
        return posibles_rutas[0][0]
    
    return None

def continuar_entrenamiento(epochs_adicionales=50, nuevo_lr=None, nuevo_batch_size=None):
    """
    Continuar el entrenamiento desde el último checkpoint (last.pt)
    
    Args:
        epochs_adicionales (int): Número de épocas adicionales
        nuevo_lr (float): Nueva tasa de aprendizaje (opcional)
        nuevo_batch_size (int): Nuevo tamaño de batch (opcional)
    """
    print("🔄 CONTINUANDO ENTRENAMIENTO")
    print("=" * 40)
    
    # Buscar el modelo last.pt
    last_model_path = encontrar_last_modelo()
    
    if last_model_path is None:
        print("❌ No se encuentra ningún modelo last.pt")
        print("💡 Asegúrate de haber entrenado un modelo anteriormente")
        return None
    
    print(f"✅ Modelo encontrado: {last_model_path}")
    
    try:
        # Cargar el modelo desde el checkpoint
        model = YOLO(str(last_model_path))
        
        # Preparar parámetros de entrenamiento
        train_params = {
            'data': 'data.yaml',
            'epochs': epochs_adicionales,
            'patience': 20,
            'save_period': 5,
            'verbose': True,
            'plots': True
        }
        
        # Agregar parámetros opcionales si se proporcionan
        if nuevo_lr is not None:
            train_params['lr0'] = nuevo_lr
            print(f"📈 Usando nueva tasa de aprendizaje: {nuevo_lr}")
        
        if nuevo_batch_size is not None:
            train_params['batch'] = nuevo_batch_size
            print(f"📦 Usando nuevo tamaño de batch: {nuevo_batch_size}")
        
        print(f"🎯 Épocas adicionales: {epochs_adicionales}")
        print(f"📊 Iniciando entrenamiento continuo...")
        print("⏱️  Esto puede tomar varios minutos dependiendo del hardware...")
        
        # Continuar entrenamiento
        results = model.train(**train_params)
        
        print("\n✅ ENTRENAMIENTO CONTINUO COMPLETADO")
        print("=" * 40)
        
        # Mostrar información del nuevo entrenamiento
        runs_dir = Path('runs/detect')
        model_dirs = [d for d in runs_dir.iterdir() if d.is_dir()]
        if model_dirs:
            latest_dir = max(model_dirs, key=lambda x: x.stat().st_mtime)
            print(f"📁 Nuevo modelo guardado en: {latest_dir}")
            
            # Verificar archivos generados
            best_path = latest_dir / 'weights' / 'best.pt'
            last_path = latest_dir / 'weights' / 'last.pt'
            
            if best_path.exists():
                print(f"🏆 Mejor modelo: {best_path}")
            if last_path.exists():
                print(f"🔄 Último checkpoint: {last_path}")
            
            # EVALUAR EL NUEVO MODELO Y MOSTRAR MÉTRICAS
            if best_path.exists():
                try:
                    print(f"\n📊 EVALUANDO NUEVO MODELO...")
                    nuevo_model = YOLO(str(best_path))
                    nuevo_metrics = nuevo_model.val()
                    
                    print(f"\n📈 MÉTRICAS DEL MODELO CONTINUADO:")
                    print(f"mAP50: {nuevo_metrics.box.map50:.3f}")
                    print(f"mAP50-95: {nuevo_metrics.box.map:.3f}")
                    print(f"Precision: {nuevo_metrics.box.mp:.3f}")
                    print(f"Recall: {nuevo_metrics.box.mr:.3f}")
                    
                    # Comparar con modelo anterior si tenemos las métricas
                    print(f"\n📊 COMPARACIÓN CON ENTRENAMIENTO ANTERIOR:")
                    print(f"{'Métrica':<12} {'Anterior':<10} {'Nuevo':<10} {'Mejora':<10}")
                    print("-" * 45)
                    
                    # Nota: Como no tenemos las métricas anteriores aquí,
                    # al menos mostramos las nuevas de forma clara
                    print(f"{'mAP50':<12} {'N/A':<10} {nuevo_metrics.box.map50:.3f:<10} {'+':<10}")
                    print(f"{'mAP50-95':<12} {'N/A':<10} {nuevo_metrics.box.map:.3f:<10} {'+':<10}")
                    print(f"{'Precision':<12} {'N/A':<10} {nuevo_metrics.box.mp:.3f:<10} {'+':<10}")
                    print(f"{'Recall':<12} {'N/A':<10} {nuevo_metrics.box.mr:.3f:<10} {'+':<10}")
                    
                except Exception as e:
                    print(f"⚠️  No se pudo evaluar el modelo: {e}")
            
            # Mostrar curvas de entrenamiento si existen
            results_png = latest_dir / 'results.png'
            if results_png.exists():
                print(f"\n📊 NUEVAS CURVAS DE ENTRENAMIENTO:")
                display(Image(str(results_png)))
            else:
                print(f"⚠️  No se encontró results.png en {latest_dir}")
        
        return model, results
        
    except Exception as e:
        print(f"❌ Error durante el entrenamiento continuo: {e}")
        return None, None

def configurar_entrenamiento_continuo():
    """Configurar y ejecutar entrenamiento continuo con opciones"""
    print("⚙️  CONFIGURACIÓN DE ENTRENAMIENTO CONTINUO")
    print("=" * 50)
    
    # Mostrar modelos disponibles
    last_model_path = encontrar_last_modelo()
    if last_model_path:
        print(f"✅ Modelo base encontrado: {last_model_path}")
        
        # Información del modelo
        try:
            model_info = YOLO(str(last_model_path))
            print(f"📋 Información del modelo:")
            print(f"   - Arquitectura: {last_model_path.parent.parent.name}")
            print(f"   - Última modificación: {datetime.fromtimestamp(last_model_path.stat().st_mtime)}")
        except:
            pass
    else:
        print("❌ No se encuentra modelo base (last.pt)")
        return
    
    print("\n🎛️  OPCIONES DE CONFIGURACIÓN:")
    print("Puedes ajustar estos parámetros según tus necesidades:")
    print("- epochs_adicionales: Número de épocas extra (default: 50)")
    print("- nuevo_lr: Nueva tasa de aprendizaje (default: mantener actual)")
    print("- nuevo_batch_size: Nuevo tamaño de batch (default: mantener actual)")
    
    print("\n📝 EJEMPLOS DE USO:")
    print("# Entrenamiento básico (50 épocas adicionales)")
    print("modelo_continuado, resultados = continuar_entrenamiento()")
    print("")
    print("# Entrenamiento con parámetros personalizados")
    print("modelo_continuado, resultados = continuar_entrenamiento(")
    print("    epochs_adicionales=100,")
    print("    nuevo_lr=0.001,")
    print("    nuevo_batch_size=8")
    print(")")
    
    return last_model_path

# Configurar entrenamiento continuo
modelo_base = configurar_entrenamiento_continuo()


# EJECUTAR ENTRENAMIENTO CONTINUO


print("\n" + "="*60)
print("🚀 INICIANDO ENTRENAMIENTO CONTINUO")
print("="*60)

# Ejecutar entrenamiento continuo con parámetros básicos
modelo_continuado, resultados = continuar_entrenamiento(
    epochs_adicionales=100,  # Cambia este número según necesites
    nuevo_lr=0.02,          # CON NONE Mantiene el learning rate actual
    nuevo_batch_size=32   # CON NONE Mantiene el batch size actual
)

# Si quieres usar parámetros personalizados, descomenta y modifica esta opción:
# modelo_continuado, resultados = continuar_entrenamiento(
#     epochs_adicionales=20,
#     nuevo_lr=0.0005,      # Learning rate más bajo para fine-tuning
#     nuevo_batch_size=8    # Batch size diferente
# )


# OPCIONES ADICIONALES (descomenta la que necesites)


# OPCIÓN 1: Entrenamiento corto para probar
# modelo_continuado, resultados = continuar_entrenamiento(epochs_adicionales=5)

# OPCIÓN 2: Entrenamiento largo
# modelo_continuado, resultados = continuar_entrenamiento(epochs_adicionales=50)

# OPCIÓN 3: Fine-tuning (learning rate más bajo)
# modelo_continuado, resultados = continuar_entrenamiento(
#     epochs_adicionales=25,
#     nuevo_lr=0.0001
# )

# OPCIÓN 4: Cambiar batch size para memoria limitada
# modelo_continuado, resultados = continuar_entrenamiento(
#     epochs_adicionales=15,
#     nuevo_batch_size=4
# )

In [ ]:
# CELDA 11: Información final y próximos pasos

print("\n" + "="*60)
print("🏁 ENTRENAMIENTO COMPLETADO")
print("="*60)

print("\n📋 RESUMEN:")
print("✅ Modelo entrenado con transfer learning")
print("✅ Evaluación completada")
print("✅ Pruebas visuales realizadas")

print("\n🔄 PRÓXIMOS PASOS:")
print("1. Revisar métricas de validación: mAP50 > 0.9 es bueno, mAP50-95 > 0.7 es excelente")
print("2. Descargar el último best.pt para usar en producción")
print("3. Si los resultados no son satisfactorios:")
print("   - Probar con modelo más grande (s, m, l, x)")
print("   - Aumentar número de épocas")
print("   - Aumentar batch_size si la GPU lo permite")
print("   - Bajar learning rate para fine-tuning")
print("   - Aumentar tamaño de imagen img_size")
print("   - modificar patience")

print("\n💡 CONSEJOS:")
print("- El archivo 'best.pt' es tu modelo final")
print("- Usa conf=0.25 como threshold inicial")
print("- Para producción, ajusta conf según tus necesidades")
print("- El modelo funciona mejor con imágenes similares al dataset")